<a href="https://colab.research.google.com/github/MMoronto/pytorch_exercises/blob/master/Peutingar_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Data Dictionary

In [3]:
pip install pytorch-forecasting

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.4/141.4 KB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.8/827.8 KB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.2/308.2 KB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.2/147.2 KB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━

##Load data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

path1='/content/drive/MyDrive/Colab Notebooks/infra_data/Salient_cement_stats.xls'
path2='/content/drive/MyDrive/Colab Notebooks/infra_data/Salient_cement_stats.csv'
fob=open(path2,)
headings=next(fob)
for rec in fob:
  Cement3=rec.split(',')
  print(Cement3)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['"SALIENT CEMENT STATISTICS1', ' 2 "', '', '', '', '', '', '', '\n']
['', '', '', '', '', '', '', '\n']
['(Thousand metric tons unless otherwise specified)', '', '', '', '', '', '', '\n']
['', '', '', '', '', '', '', '\n']
['', '', '', '2014', '2015', '2016', '2017', '2018\n']
['United States:', '', '', '', '', '', '', '\n']
['Production:', '', '', '', '', '', '', '\n']
['Cement3', '', '', '"82', '535"', '"84', '405"', '"84', '695"', '"86', '356"', '"86', '400"\n']
['Clinker', '', '', '"74', '372"', '"76', '043"', '"75', '633"', '"76', '678"', '"77', '112"\n']
['"Shipments from mills and terminals:3', ' 4"', '', '', '', '', '', '', '\n']
['Quantity', '', '', '"88', '900"', '"92', '000"', '"94', '300"', '"96', '900"', '"98', '500"\n']
['Value5', 'thousand dollars', '', '"8', '940', '000"', '"9', '800', '000"', '"10', '500', '000"', '"11', '300', '000"', '"11'

In [5]:
import os
import warnings

warnings.filterwarnings("ignore")  # avoid printing out absolute paths

os.chdir("../../..")

import copy
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters


In [6]:
# Create a series and let pandas create a default integer index:
s = pd.Series([1, 3, 5, np.nan, 6, 8])
s

0    1.0
1    3.0
2    5.0
3    NaN
4    6.0
5    8.0
dtype: float64

In [7]:
from pandas.io.stata import DatetimeIndex
# Create a DataFrame by passing a NumPy array with a dateline index using `date_range()` & labeled columns:
dates = pd.date_range("20230101", periods=8)

dates

# DatetimeIndex(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04', '2013-01-05', '2013-01-06'], dtype='datatime64[ns]', freq='D')

df = pd.DataFrame(np.random.randn(8, 6), index=dates, columns=list("ABCDEF"))

df 

,A,B,C,D,E,F
2023-01-01,0.009180,0.745143,-1.024358,0.331243,-0.703555,0.373640
2023-01-02,-1.601455,1.764950,-0.643015,-0.346160,0.773347,0.003524
2023-01-03,-0.548832,0.543325,0.254254,-0.617908,1.765905,0.018999
2023-01-04,-0.905954,1.371833,-0.698850,-0.844793,0.722521,0.463176
2023-01-05,0.479406,0.544400,-0.448426,0.723683,-0.491147,1.742837
2023-01-06,-1.976807,-0.808416,0.566530,0.868808,0.827001,0.015793
2023-01-07,0.193658,-0.123459,0.480813,-1.227733,-0.206619,-0.187632
2023-01-08,1.663893,0.108888,1.121314,-0.568425,1.667642,-0.030761


In [8]:
# Create a `DataFrame` by passing a dictionary of objects that can be converted into a series-like structure:

df2 = pd.DataFrame(
    {
        "A": 1.0,
        "B": pd.Timestamp("20130102"),
        "C": pd.Series(1, index=list(range(4)), dtype="float32"),
        "D": np.array([3] * 4, dtype="int32"),
        "E": pd.Categorical(["test", "train", "test", "train"]),
        "F": "foo",
    }
)

df2

,A,B,C,D,E,F
0,1.0,2013-01-02,1.0,3,test,foo
1,1.0,2013-01-02,1.0,3,train,foo
2,1.0,2013-01-02,1.0,3,test,foo
3,1.0,2013-01-02,1.0,3,train,foo


In [9]:
df2.dtypes

A           float64
B    datetime64[ns]
C           float32
D             int32
E          category
F            object
dtype: object

In [10]:
df2.D

0    3
1    3
2    3
3    3
Name: D, dtype: int32

##Viewing data

In [11]:
df.head()

,A,B,C,D,E,F
2023-01-01,0.009180,0.745143,-1.024358,0.331243,-0.703555,0.373640
2023-01-02,-1.601455,1.764950,-0.643015,-0.346160,0.773347,0.003524
2023-01-03,-0.548832,0.543325,0.254254,-0.617908,1.765905,0.018999
2023-01-04,-0.905954,1.371833,-0.698850,-0.844793,0.722521,0.463176
2023-01-05,0.479406,0.544400,-0.448426,0.723683,-0.491147,1.742837


In [12]:
df.tail()

,A,B,C,D,E,F
2023-01-04,-0.905954,1.371833,-0.698850,-0.844793,0.722521,0.463176
2023-01-05,0.479406,0.544400,-0.448426,0.723683,-0.491147,1.742837
2023-01-06,-1.976807,-0.808416,0.566530,0.868808,0.827001,0.015793
2023-01-07,0.193658,-0.123459,0.480813,-1.227733,-0.206619,-0.187632
2023-01-08,1.663893,0.108888,1.121314,-0.568425,1.667642,-0.030761


In [13]:
df.columns

Index(['A', 'B', 'C', 'D', 'E', 'F'], dtype='object')

In [14]:
df.index

DatetimeIndex(['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04',
               '2023-01-05', '2023-01-06', '2023-01-07', '2023-01-08'],
              dtype='datetime64[ns]', freq='D')

`DataFrame.to_numpy()` gives a NumPy representation of the underlying data. 

In [15]:
df.to_numpy()

array([[ 0.00917987,  0.74514315, -1.02435842,  0.33124333, -0.70355479,
         0.37364045],
       [-1.60145502,  1.76495037, -0.64301484, -0.34616026,  0.77334652,
         0.00352392],
       [-0.54883238,  0.54332526,  0.25425437, -0.61790767,  1.76590516,
         0.01899897],
       [-0.90595435,  1.37183336, -0.69884972, -0.84479345,  0.72252127,
         0.46317553],
       [ 0.47940607,  0.54439969, -0.44842597,  0.72368265, -0.49114707,
         1.74283672],
       [-1.97680731, -0.80841566,  0.56652988,  0.86880784,  0.82700134,
         0.01579327],
       [ 0.19365771, -0.1234588 ,  0.48081277, -1.22773287, -0.20661875,
        -0.18763197],
       [ 1.6638934 ,  0.10888788,  1.12131354, -0.56842454,  1.66764234,
        -0.03076055]])

In [16]:
df2.to_numpy()

array([[1.0, Timestamp('2013-01-02 00:00:00'), 1.0, 3, 'test', 'foo'],
       [1.0, Timestamp('2013-01-02 00:00:00'), 1.0, 3, 'train', 'foo'],
       [1.0, Timestamp('2013-01-02 00:00:00'), 1.0, 3, 'test', 'foo'],
       [1.0, Timestamp('2013-01-02 00:00:00'), 1.0, 3, 'train', 'foo']],
      dtype=object)

In [17]:
df.describe()

,A,B,C,D,E,F
count,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000
mean,-0.335864,0.518333,-0.048967,-0.210161,0.544387,0.299947
std,1.178856,0.817231,0.756609,0.763514,0.934385,0.622053
min,-1.976807,-0.808416,-1.024358,-1.227733,-0.703555,-0.187632
25%,-1.079830,0.050801,-0.656974,-0.674629,-0.277751,-0.005047
50%,-0.269826,0.543862,-0.097086,-0.457292,0.747934,0.017396
75%,0.265095,0.901816,0.502242,0.429353,1.037162,0.396024
max,1.663893,1.764950,1.121314,0.868808,1.765905,1.742837


In [20]:
df

,A,B,C,D,E,F
2023-01-01,0.009180,0.745143,-1.024358,0.331243,-0.703555,0.373640
2023-01-02,-1.601455,1.764950,-0.643015,-0.346160,0.773347,0.003524
2023-01-03,-0.548832,0.543325,0.254254,-0.617908,1.765905,0.018999
2023-01-04,-0.905954,1.371833,-0.698850,-0.844793,0.722521,0.463176
2023-01-05,0.479406,0.544400,-0.448426,0.723683,-0.491147,1.742837
2023-01-06,-1.976807,-0.808416,0.566530,0.868808,0.827001,0.015793
2023-01-07,0.193658,-0.123459,0.480813,-1.227733,-0.206619,-0.187632
2023-01-08,1.663893,0.108888,1.121314,-0.568425,1.667642,-0.030761


In [18]:
df.T

,2023-01-01,2023-01-02,2023-01-03,2023-01-04,2023-01-05,2023-01-06,2023-01-07,2023-01-08
A,0.009180,-1.601455,-0.548832,-0.905954,0.479406,-1.976807,0.193658,1.663893
B,0.745143,1.764950,0.543325,1.371833,0.544400,-0.808416,-0.123459,0.108888
C,-1.024358,-0.643015,0.254254,-0.698850,-0.448426,0.566530,0.480813,1.121314
D,0.331243,-0.346160,-0.617908,-0.844793,0.723683,0.868808,-1.227733,-0.568425
E,-0.703555,0.773347,1.765905,0.722521,-0.491147,0.827001,-0.206619,1.667642
F,0.373640,0.003524,0.018999,0.463176,1.742837,0.015793,-0.187632,-0.030761


In [21]:
df.sort_index(axis=1, ascending=False)

,F,E,D,C,B,A
2023-01-01,0.373640,-0.703555,0.331243,-1.024358,0.745143,0.009180
2023-01-02,0.003524,0.773347,-0.346160,-0.643015,1.764950,-1.601455
2023-01-03,0.018999,1.765905,-0.617908,0.254254,0.543325,-0.548832
2023-01-04,0.463176,0.722521,-0.844793,-0.698850,1.371833,-0.905954
2023-01-05,1.742837,-0.491147,0.723683,-0.448426,0.544400,0.479406
2023-01-06,0.015793,0.827001,0.868808,0.566530,-0.808416,-1.976807
2023-01-07,-0.187632,-0.206619,-1.227733,0.480813,-0.123459,0.193658
2023-01-08,-0.030761,1.667642,-0.568425,1.121314,0.108888,1.663893


In [22]:
df.sort_values(by="B")

,A,B,C,D,E,F
2023-01-06,-1.976807,-0.808416,0.566530,0.868808,0.827001,0.015793
2023-01-07,0.193658,-0.123459,0.480813,-1.227733,-0.206619,-0.187632
2023-01-08,1.663893,0.108888,1.121314,-0.568425,1.667642,-0.030761
2023-01-03,-0.548832,0.543325,0.254254,-0.617908,1.765905,0.018999
2023-01-05,0.479406,0.544400,-0.448426,0.723683,-0.491147,1.742837
2023-01-01,0.009180,0.745143,-1.024358,0.331243,-0.703555,0.373640
2023-01-04,-0.905954,1.371833,-0.698850,-0.844793,0.722521,0.463176
2023-01-02,-1.601455,1.764950,-0.643015,-0.346160,0.773347,0.003524


##Getting

In [28]:
# Select a single column, which yields a Series, equivalent to 'df.A'
df["A"]

2023-01-01    0.009180
2023-01-02   -1.601455
2023-01-03   -0.548832
2023-01-04   -0.905954
2023-01-05    0.479406
2023-01-06   -1.976807
2023-01-07    0.193658
2023-01-08    1.663893
Freq: D, Name: A, dtype: float64

In [33]:
df[0:3]

,A,B,C,D,E,F
2023-01-01,0.009180,0.745143,-1.024358,0.331243,-0.703555,0.373640
2023-01-02,-1.601455,1.764950,-0.643015,-0.346160,0.773347,0.003524
2023-01-03,-0.548832,0.543325,0.254254,-0.617908,1.765905,0.018999


##Selection by label

In [35]:
df.loc[dates[1]]

A   -1.601455
B    1.764950
C   -0.643015
D   -0.346160
E    0.773347
F    0.003524
Name: 2023-01-02 00:00:00, dtype: float64

In [37]:
#Selecting on a multi-axis by label:
df.loc[:, ["A", "D"]]

,A,D
2023-01-01,0.009180,0.331243
2023-01-02,-1.601455,-0.346160
2023-01-03,-0.548832,-0.617908
2023-01-04,-0.905954,-0.844793
2023-01-05,0.479406,0.723683
2023-01-06,-1.976807,0.868808
2023-01-07,0.193658,-1.227733
2023-01-08,1.663893,-0.568425


In [42]:
# Showing label slicing, both endpoints are included:
df.loc["20230104":"20230106", ["A", "D"]]

,A,D
2023-01-04,-0.905954,-0.844793
2023-01-05,0.479406,0.723683
2023-01-06,-1.976807,0.868808


In [43]:
# Reduction in the dimensions of the returned object:
df.loc["20230104", ["A", "D"]]

A   -0.905954
D   -0.844793
Name: 2023-01-04 00:00:00, dtype: float64

In [45]:
# For getting a scalar value
df.loc[dates[1], "D"]

-0.34616025977783066

In [46]:
# For getting fast access to a scalar value
df.at[dates[1], "D"]

-0.34616025977783066

##Selection by position

In [48]:
df.iloc[3]

A   -0.905954
B    1.371833
C   -0.698850
D   -0.844793
E    0.722521
F    0.463176
Name: 2023-01-04 00:00:00, dtype: float64